# Clustering of Luxembourg

## PROBLEM STATEMENT

    In this work we are going to conduct cluster analysis of Luxembourg. It is a small country in the western Europe, landlocked between Belgium, France and Germany. Its population amounts to 590 000 as of 1- Jan-2017. Despite of the small size, it is quite a developed country and it attracts lot of foreigners, such as tourists, expats and cross-border workers. 
    We will collect information about Luxembourg communities, draw these on the map and conduct cluster analysis based on venues in these communities, in order to classify them such as Oher business/tourist area, or touristic/leisure area, or suburb/dormitory area, etc. Having the clusters reflected on a map, it would be easier for one to understand where to go for a weekend, or for a longer rest, or stay during a business trip. Also, the obtained information could provide a view to quickly understand what area of the country or even a community would represent an interest for a visit/trip/stay/work/live.


## DATA USED, AND THE APPROACH APPLIED

    For the analysis, we use data files publicly available on Luxembourg data portal. The file exposes information about all the communities in Luxembourg, their population as of 1/1/2017, and geolocation. Also, in order to repartition the largest city, Ville de Luxembourg, on districts, in order to have this comparable with smaller communities, we will use another publicly available data file shared on the same data portal. Additionally, we would need to use Geocoder API in order to obtain geolocations of the districts. 
    We are going to use Foursquare API for the determination of venues available in the locations and nearby (in radius 3 km).  Having this information, we will conduct cluster analysis of the communities and, as a result, get different clusters of Luxembourg communities mapped in on a chart in different colors. We will also analyze most used venues in the clusters and will try to provide description of distinctions among the clusters.


#### Load communities geolocation from publicly open dataset

In [176]:
import pandas as pd
import requests
import numpy as np

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
# The code was removed by Watson Studio for sharing.

,NOM_COMMUNE_ADMINISTRATIVE,NOM_COMMUNE_CADASTRALE,NOM_OFFICIEL_LOCALITE,LOCALITE_OFFICIELLE,NOM_LUXEMBOURGEOIS (svt. CPLL),Y_RECHTS,X_HOCH,LON_LL84,LAT_LL84,Unnamed: 9,localité officielle = issue du registre national des localités et des rues,CPLL = Conseil permanent de la langue luxembourgeoise
0,Bettembourg,Bettembourg,Abweiler,OUI,Obeler,73578.0,66589.0,6.079404,49.533972,NaN,NaN,NaN
3,Wormeldange,Wormeldange,Ahn,OUI,Ohn,98328.0,76929.0,6.421799,49.626693,NaN,NaN,NaN
6,Wincrange,Oberwampach,Allerborn,OUI,Allerbuer,58773.0,122538.0,5.871837,50.036652,NaN,NaN,NaN
9,Kiischpelt,Kautenbach,Alscheid,OUI,Alschent,68385.0,115157.0,6.006226,49.970558,NaN,NaN,NaN
10,Junglinster,Junglinster,Altlinster,OUI,Allënster,83286.0,87411.0,6.213694,49.721206,NaN,NaN,NaN


In [ ]:
# The code was removed by Watson Studio for sharing.

In [5]:
# The code was removed by Watson Studio for sharing.

,LOCALITY,LAT_LL84,LON_LL84
0,Abweiler,49.533972,6.079404
3,Ahn,49.626693,6.421799
6,Allerborn,50.036652,5.871837
9,Alscheid,49.970558,6.006226
10,Altlinster,49.721206,6.213694


In [6]:
# The code was removed by Watson Studio for sharing.

,LOCALITY,LAT_LL84,LON_LL84
984,Wolper,49.765362,6.354330
985,Wolwelange,49.828933,5.766363
986,Wormeldange,49.611944,6.402627
987,Wormeldange-Haut,49.611944,6.402627
988,Zittig,49.745253,6.346538


In [8]:
# The code was removed by Watson Studio for sharing.

The number of communities is  565


In [110]:
!wget -q -O "lux_population.xls" https://data.public.lu/en/datasets/r/1287230d-53fa-451b-bf62-5aa55c3c2a20
luxpop = pd.read_excel(io="lux_population.xls", sheet_name ="A", skiprows=3)

In [111]:
luxpop.head()

,Code LAU2,Unnamed: 1,Commune,1-1-2000,1-1-2001,15-02-2001,1-1-2002,1-1-2003,1-1-2004,1-1-2005,...,1-1-2009,1-1-2010,1-1-2011,1-2-2011,1-1-2012,1-1-2013,1-1-2014,1-1-2015,1-1-2016,1-1-2017
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,304.0,NaN,Luxembourg-Ville,80670.0,81804.0,76688.0,77965.0,78329.0,79366.0,80158.0,...,88586.0,90848.0,94034.0,95058.0,99852.0,103641.0,107247.0,111287.0,115227.0,114303.0
2,101.0,NaN,Dippach,3136.0,3209.0,3189.0,3233.0,3234.0,3332.0,3417.0,...,3469.0,3537.0,3564.0,3544.0,3627.0,3727.0,3859.0,4037.0,4071.0,4178.0
3,102.0,NaN,Garnich,1504.0,1485.0,1495.0,1511.0,1491.0,1502.0,1537.0,...,1718.0,1771.0,1802.0,1861.0,1895.0,1931.0,1943.0,1991.0,2043.0,2099.0
4,103.0,NaN,Hobscheid,2512.0,2502.0,2563.0,2579.0,2609.0,2665.0,2720.0,...,2982.0,3026.0,3079.0,3096.0,3173.0,3239.0,3271.0,3397.0,3434.0,3518.0


In [112]:
#leave only columns with community names and population as of 1-1-2017
#remove last row with totals

luxpop = luxpop.iloc[1:-2, [2, -1]]
luxpop.columns = ["Community", "Population"]

In [113]:
luxpop.tail()

,Community,Population
101,Mondorf-les-Bains,4999.0
102,Remich,3632.0
103,Schengen,4805.0
104,Stadtbredimus,1866.0
105,Waldbredimus,1098.0


In [114]:
print("The number of communities is ", luxpop.shape[0])

The number of communities is  105


In [31]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# import k-means from clustering stage
from sklearn.cluster import KMeans

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   1.28 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  31.60 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  35.19 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  36.51 MB/s


In [115]:
# The code was removed by Watson Studio for sharing.

    We notice that Luxembourg-ville is the largest city in the country, and we want to split it on the districts in our analysis, in order to consider each district separately as this could improve quality of our examination. Data for the Luxembourg-ville are also available in luxembourg data portal.

In [116]:
url = project.get_file("popcom2000-2017_LAU2.xlsx")
vdldata=pd.read_excel(io=url, sheet_name="Luxembourg-ville")
vdldata = vdldata.iloc[0:-1, 0:2]
vdldata.columns = ["Community", "Population"]

In [117]:
vdldata.head()

,Community,Population
0,BEGGEN,3455
1,BELAIR,11285
2,BONNEVOIE-NORD/VERLORENKOST,4172
3,BONNEVOIE-SUD,12429
4,CENTS,6262


In [118]:
print("Number of Luxembourg-ville districts is", vdldata.shape[0])

Number of Luxembourg-ville districts is 24


We will join Luxembourg communities and districts of Luxembourg-ville

In [119]:
luxpop=luxpop.append(vdldata, ignore_index=True)
luxpop=luxpop[luxpop.Community != "Luxembourg-Ville"]

In [175]:
print("Total number of communities is ", luxpop.shape[0])

Total number of communities is  128


In [120]:
# The code was removed by Watson Studio for sharing.

We add geolocation coordinates to each of the communities, making use of *geopy.geocoders* library

In [ ]:
geolocator = Nominatim(user_agent="my-application1")
luxpop["Latitude"] = ""
luxpop["Longtitude"] = ""

for idx in luxpop.index:
    address = luxpop.loc[idx, "Community"] + ", LU"
    #print(address)
    loc1 = geolocator.geocode(address)
    lat1 = loc1.latitude
    lon1 = loc1.longitude
    luxpop.loc[idx, "Latitude"] = lat1
    luxpop.loc[idx, "Longtitude"] = lon1


In [158]:
# The code was removed by Watson Studio for sharing.

### Map Luxembourg comminities

In [170]:
# The code was removed by Watson Studio for sharing.

In [171]:
address = 'Luxembourg'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Luxembourg is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Luxembourg is 49.8158683, 6.1296751.


In [177]:
map_lux = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, community, population in zip(luxpop['Latitude'], luxpop['Longtitude'], luxpop['Community'], luxpop["Population"]):
    label = '{} {}'.format(community, population)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lux)  
    
map_lux